In [1]:
# | default_exp routes.group

In [2]:
# | exporti
import httpx
from enum import Enum
from typing import Union

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

# Search and Get Routes


In [3]:
# | export
class SearchGroups_Error(de.DomoError):
    def __init__(
        self, status, message, domo_instance, function_name="search_groups_by_name"
    ):
        super().__init__(
            function_name=function_name,
            status=status,
            message=message,
            domo_instance=domo_instance,
        )


async def search_groups_by_name(
    auth: dmda.DomoAuth,
    search_name: str,
    is_exact_match: bool = True,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """uses /content/v2/groups/grouplist api -- includes user details"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist?ascending=true&search={search_name}&sort=name "

    res = await gd.get_data(
        auth=auth, url=url, method="GET", debug_api=debug_api, session=session
    )
    if not is_exact_match:
        return res

    match_group = next(
        (group for group in res.response if group.get("name") == search_name), None
    )
    # print(match_group)

    if not match_group:
        raise SearchGroups_Error(
            status=res.status,
            message=f"There is no exact match for {search_name}",
            domo_instance=auth.domo_instance,
        )
    res.response = match_group

    return res

#### Sample implementation of search_groups_by_name


In [4]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
search_name = "Test Group 2"
try:
    res = await search_groups_by_name(
        auth=token_auth, search_name=search_name, debug_api=False
    )
    print(res)
except SearchGroups_Error as e:
    print(e)
except Exception as e:
    print(e)

ResponseGetData(status=200, response={'name': 'Test Group 2', 'groupId': 1259653287, 'owners': [{'type': 'GROUP', 'id': '822382906', 'displayName': 'Grant: Manage all groups'}, {'type': 'USER', 'id': '1893952720', 'displayName': 'Jae Wilson1'}, {'type': 'GROUP', 'id': '1259653287', 'displayName': 'Test Group 2'}, {'type': 'GROUP', 'id': '2146122228', 'displayName': 'test'}, {'type': 'USER', 'id': '1681443709', 'displayName': 'test 3'}], 'groupType': 'open', 'groupMembers': [{'type': 'USER', 'id': '2072616249'}, {'type': 'USER', 'id': '663516735'}], 'memberCount': 2, 'created': '2023-08-17 14:25:33.0', 'description': 'update metadata - updated 2023-08-17'}, is_success=True)


In [5]:
# | export


async def get_all_groups(
    auth: dmda.DomoAuth, debug_api: bool = False, session: httpx.AsyncClient = None
) -> rgd.ResponseGetData:
    """uses /content/v2/groups/grouplist api -- includes user details"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist"

    res = await gd.get_data(
        url=url, method="GET", auth=auth, debug_api=debug_api, session=session
    )

    return res

#### Sample implementation of get_all_groups


In [6]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_all_groups(auth=token_auth)
res_df = pd.DataFrame(res.response)
res_df[0:1]

,name,groupId,owners,groupType,groupMembers,memberCount,created,description
0,ADM | Orientation,49793884,"[{'type': 'GROUP', 'id': '49793884', 'displayN...",open,"[{'type': 'USER', 'id': '908494860'}, {'type':...",15,2023-06-08 20:59:05.0,


In [7]:
# | export
async def get_group_by_id(
    auth: dmda.DomoAuth,
    group_id: str,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    """uses /content/v2/groups/ api -- does not return details"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/{group_id}"

    res = await gd.get_data(
        auth=auth, url=url, method="GET", debug_api=debug_api, session=session
    )

    if res.status == 404 and res.response == "Not Found":
        raise SearchGroups_Error(
            status=res.status,
            message=f"group {group_id} not found",
            domo_instance=auth.domo_instance,
            function_name="get_group_by_id",
        )

    return res

In [8]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_group_by_id(auth=token_auth, group_id=1259653287)

ResponseGetData(status=200, response={'id': 1259653287, 'name': 'Test Group 2', 'type': 'open', 'userIds': [2072616249, 663516735], 'creatorId': 1893952720, 'memberCount': 2, 'guid': 'aabb6587-ad53-11ed-82c3-0a09ba383c95', 'description': 'update metadata - updated 2023-08-17', 'hidden': False, 'default': False, 'active': True}, is_success=True)

# CRUD Routes


In [9]:
# | export
class GroupType_Enum(Enum):
    OPEN = "open"
    ADHOC = "adHoc"
    CLOSED = "closed"
    DIRECTORY = "directory"
    DYNAMIC = "dynamic"
    SYSYTEM = "system"


def generate_body_create_group(
    group_name: str, group_type: str = "open", description: str = ""
) -> dict:
    """Generates the body to create group for content_v2_group API"""
    body = {"name": group_name, "type": group_type, "description": description}

    return body

#### sample_implementation of generate_body_create_group


In [10]:
generate_body_create_group(
    group_name="test_group_name",
    group_type=GroupType_Enum.ADHOC.value,
    description="from jupyter",
)

{'name': 'test_group_name', 'type': 'adHoc', 'description': 'from jupyter'}

In [11]:
# | export
class CreateGroup_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name="create_group"):
        super().__init__(
            function_name=function_name,
            status=status,
            message=message,
            domo_instance=domo_instance,
        )


async def create_group(
    auth: dmda.DomoAuth,
    group_name: str,
    group_type: str = "open",
    description: str = "",
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    # body : {"name": "GROUP_NAME", "type": "open", "description": ""}

    body = generate_body_create_group(
        group_name=group_name, group_type=group_type, description=description
    )

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    if not res.is_success:
        group_exists = await search_groups_by_name(
            auth=auth, search_name=group_name, is_exact_match=True
        )
        if group_exists.is_success:
            raise CreateGroup_Error(
                status=res.status,
                message=f"{group_name} already exists. Choose a different group_name",
                domo_instance=auth.domo_instance,
                function_name="create_group",
            )

    if not res.is_success:
        raise CreateGroup_Error(
            status=res.status,
            message=res.response,
            domo_instance=auth.domo_instance,
            function_name="create_group",
        )

    return res

#### Sample implementation of create_group


In [12]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
try:
    res = await create_group(
        auth=token_auth, group_name="Test Group ABC", debug_api=False
    )
    res.response

except CreateGroup_Error as e:
    print(e)

create_group: Status 400 - Test Group ABC already exists. Choose a different group_name at domo-community


In [13]:
# | export
async def update_group(
    auth: dmda.DomoAuth,
    group_id: int,
    group_name: str = None,
    group_type: str = None,
    description: str = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    body = [
        {
            "groupId": int(group_id),
            "name": group_name,
            "type": group_type,
            "description": description,
        }
    ]

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of update_group


In [14]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 250707376

await update_group(
    auth=token_auth,
    group_id=group_id,
    group_name="Test Group ABC",
    description=f"updated via API on {dt.date.today()}",
    debug_api=False,
)

ResponseGetData(status=200, response='', is_success=True)

# Group Membership
## GET  Group Ownership

In [15]:
# | export
async def get_group_owners(
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    # url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/access"
    # url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/users?group={group_id}"
    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/permissions?checkOwnership=true&includeUsers=false'

    res = await gd.get_data(
        auth=auth,
        url=url,
        body = [str(group_id)],
        method="POST",
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    res.response = res.response[0].get('permissions').get('owners')
    return res



#### sample implementation of get_ownership

In [16]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 1781661643

res = await get_group_owners(auth=token_auth,
                           group_id=group_id,
                           )

pd.DataFrame(res.response)

,type,id,displayName
0,GROUP,822382906,Grant: Manage all groups
1,GROUP,1781661643,TestMembership
2,USER,1893952720,Jae Wilson1


In [17]:
# | export
async def get_group_membership(
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    # url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/access"
    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/users?group={group_id}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    res.response = res.response.get('groupUserList')
    return res

#### sample implementation of get_membership

In [18]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 1781661643

res = await get_group_membership(auth=token_auth,
                           group_id=group_id,
                           )

pd.DataFrame(res.response)

,userId,role,roleId,lastSignIn,location
0,696468809,Community_Default_Priviliged,2097317660,2021-04-19 16:36:26.944,NaN
1,1345737456,Privileged,2,2023-06-23 04:12:09.0,
2,1141078945,Community_Default_Priviliged,2097317660,2021-04-28 03:58:56.0,NaN


## CRUD Group Membership

In [19]:
#| exporti
def generate_body_update_group_membership_entity(user_id: Union[str, int],
                                                 user_type: str  # USER or GROUP
                                                 ):
    if user_type == 'USER':
        return {"type": "USER", "id": str(user_id)}
    elif user_type == 'GROUP':
        return {"type": "GROUP", "id": int(user_id)}


In [20]:
# | export
def generate_body_update_group_membership(group_id: str,
                                          add_member_arr: list[str] = None,
                                          remove_member_arr: list[str] = None,

                                          add_owner_arr: list[str] = None,
                                          remove_owner_arr: list[str] = None) -> list[dict]:
    """
    each member or owner obj should be an object of shape {"type", "id"}
    """

    body = {"groupId": int(group_id)}

    if add_owner_arr and len(add_owner_arr) > 0 :
        body.update({"addOwners": [generate_body_update_group_membership_entity(
            user_id=obj.get('id'), user_type=obj.get('type')) for obj in add_owner_arr]})

    if remove_owner_arr and len(remove_owner_arr) > 0:
        body.update({"removeOwners": [generate_body_update_group_membership_entity(
            user_id=obj.get('id'), user_type=obj.get('type')) for obj in remove_owner_arr]})

    if remove_member_arr and len(remove_member_arr) > 0:
        body.update({"removeMembers": [
                    generate_body_update_group_membership_entity(user_id=obj.get('id'), user_type=obj.get('type')) for obj in remove_member_arr]})
    if add_member_arr and len(add_member_arr) > 0:
        body.update(
            {"addMembers": [generate_body_update_group_membership_entity(user_id=obj.get('id'), user_type=obj.get('type')) for obj in add_member_arr]})

    return [body]


In [21]:
# | export
async def update_group_membership(
    auth: dmda.DomoAuth,
    group_id: str,
    add_member_arr: list[dict] = None,
    remove_member_arr: list[dict] = None,
    add_owner_arr: list[dict] = None,
    remove_owner_arr: list[dict] = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    """
    each member or owner obj should be an object of shape {"type", "id"}
    """

    body = generate_body_update_group_membership(group_id = group_id,
                                                 add_member_arr = add_member_arr,
                                                 remove_member_arr= remove_member_arr,
                                                 add_owner_arr = add_owner_arr,
                                                 remove_owner_arr = remove_owner_arr)

    # body = [{
    #     "groupId":"GROUP_ID",
    #     "removeMembers": [{"type":"USER","id":"USER_ID"}],
    #     "addMembers"   : [{"type":"USER","id":"USER_ID"}]
    # }]
    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/access"

    if debug_api:
        print(url, body)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res


#### Sample implementation of generate_body_update_group_membership and update_group_membership


In [22]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 250707376

add_user_obj = {'id':  os.environ["DOMO_DOJO_USER_ID"],
                'type': 'USER'}

await update_group_membership(auth=token_auth, 
                        group_id=group_id,
                                    add_member_arr=[add_user_obj], 
                                    add_owner_arr=[add_user_obj])


ResponseGetData(status=200, response='', is_success=True)

In [23]:
# | hide
import nbdev

nbdev.nbdev_export()